In [1]:
is_mac = False

if is_mac == True:
    baseprompt_filepath = "Data/basePrompt.in"
    instagram_data_filepath = "Data/instagram_raw.csv"
    output_file = "/Users/madhavmalik/VSC Projects/uw-upnext/uw-upnext/public/events.json"
    image_filepath = '/Users/madhavmalik/VSC Projects/uw-upnext/uw-upnext/public/InstagramImages/'
else:
    baseprompt_filepath = "Data/basePrompt.in"
    instagram_data_filepath = "Data/instagram_raw.csv"
    output_file = "../public/events.json"
    image_filepath = "../public/InstagramImages/"


In [46]:
import pandas as pd
import datetime 
import time
from requests.exceptions import RequestException
from typing import List
import json
from together import Together
import os
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import json
import pandas as pd
from enum import Enum, auto
import re
from pymongo import MongoClient, errors
from pymongo.server_api import ServerApi
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

basePrompt = open(baseprompt_filepath,"r", encoding = "utf-8").read()

load_dotenv()
togetherAPI = os.getenv('TOGETHER_API')

In [47]:

#Updating WUSA Events
webpage = requests.get("https://wusa.ca/events/")
jsonscript =str(webpage.content)
isolatedinformation=jsonscript.split('<script type="application/ld+json">')[1].split("</script>")[0][4:-4].encode("utf16", errors="surrogatepass").decode("utf16").encode().decode('unicode_escape')

WUSAjson = json.loads(isolatedinformation.replace('&lt;p&gt;',"").replace("[&hellip;]&lt;/p&gt;\\\\n",""))

wusaDf ={}
postscolumns = ['account','date','caption',"display_photo",'event_details']
wusaDf = pd.DataFrame(columns = postscolumns)

import time

time_now = int(time.time())
for event in WUSAjson:
    if int(datetime.datetime.fromisoformat(event["startDate"]).timestamp()) >= time_now:
        try: 
            location = event["location"]["address"]["streetAddress"]
        except Exception as err:
            print("No street address", str(err))
            location = None
        try:
            new_row = pd.DataFrame({
                    "account": ["WUSA"],
                    "date": [time_now],
                    "caption": str(event["description"])+" [For More Information, Click View Post] ",
                    "display_photo": event["image"],
                    "url": [event['url']],
                    "likes": [0],
                    "event_details": [{
                        "is_event": True,
                        "event_name": event["name"],
                        "event_description": str(event["description"])+" ... ",
                        "categories": ["SOCIAL"],
                        "start_time": int(datetime.datetime.fromisoformat(event["startDate"]).timestamp()),
                        "end_time": int(datetime.datetime.fromisoformat(event["endDate"]).timestamp()),
                        "location": location,
                    }]
            })
            wusaDf = pd.concat([wusaDf, new_row])
            print("success")
        except Exception as error:
            print("Failed row, error:", error)

success
success
success
success
success
success
success
success
success
success
success
Failed row, error: 'image'
success
No street address 'streetAddress'
Failed row, error: 'image'
success
No street address 'streetAddress'
success
No street address 'streetAddress'
Failed row, error: 'image'
No street address 'streetAddress'
success
success
No street address 'location'
success
Failed row, error: 'image'
No street address 'streetAddress'
success
success
No street address 'location'
success
success
success


In [48]:
together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'WhereIsAI/UAE-Large-V1' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

insertObjectIds = []

# Set up MongoDB client
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

# Remove all documents with 'account': "WUSA"
try:
    result = collection.delete_many({'account': "WUSA"})
    print(f"Documents removed: {result.deleted_count}")
except errors.PyMongoError as e:
    print(f"Error deleting documents: {e}")

# Check and remove '_id' column if present
if '_id' in wusaDf.columns:
    print("DataFrame contains an '_id' column. It will be removed to prevent duplication.")
    wusaDf = wusaDf.drop(columns=['_id'])

for index, row in wusaDf.iterrows():
    # Construct the info string
    info = f'"id": "{int(index)}"|* "account": "{row["account"]}"|* "date": "{row["date"]}"|* "caption": "{row["caption"]}"|*'
    embeddedtext = ',\n'.join(x for x in info.replace('\n','\\n').split('|*')) 

    # Create the document dictionary
    row_dict = {column: row[column] for column in wusaDf.columns.tolist()}
    
    # Add the embedded field
    row_dict["embedded"] = generate_embedding([embeddedtext], embedding_model_string)
    
    # Remove '_id' if present to let MongoDB generate it
    row_dict.pop('_id', None)
    
    try:
        result = collection.insert_one(row_dict)
        print(f"Inserted document ID: {result.inserted_id}")
        insertObjectIds.append(result.inserted_id)
    except errors.DuplicateKeyError as e:
        print(f"DuplicateKeyError: {e}. Document skipped.")
    except errors.PyMongoError as e:
        print(f"An error occurred: {e}. Document skipped.")
    
    time.sleep(0.5)  # Adjust or remove delay as needed

# Optionally, print all inserted ObjectIds
print(f"All inserted document IDs: {insertObjectIds}")

Documents removed: 22
Inserted document ID: 672979b038120b04865491bd
Inserted document ID: 672979b138120b04865491be
Inserted document ID: 672979b238120b04865491bf
Inserted document ID: 672979b238120b04865491c0
Inserted document ID: 672979b338120b04865491c1
Inserted document ID: 672979b438120b04865491c2
Inserted document ID: 672979b538120b04865491c3
Inserted document ID: 672979b638120b04865491c4
Inserted document ID: 672979b738120b04865491c5
Inserted document ID: 672979b738120b04865491c6
Inserted document ID: 672979b838120b04865491c7
Inserted document ID: 672979b938120b04865491c8
Inserted document ID: 672979ba38120b04865491c9
Inserted document ID: 672979bb38120b04865491ca
Inserted document ID: 672979bc38120b04865491cb
Inserted document ID: 672979bc38120b04865491cc
Inserted document ID: 672979bd38120b04865491cd
Inserted document ID: 672979be38120b04865491ce
Inserted document ID: 672979bf38120b04865491cf
Inserted document ID: 672979c038120b04865491d0
Inserted document ID: 672979c138120b04

In [74]:
#Validating if it is an event
load_dotenv()
togetherAPI = os.getenv('TOGETHER_API')
client = Together(api_key=togetherAPI)

postsDf = pd.read_csv(filepath_or_buffer=instagram_data_filepath).replace('"','', regex=True).iloc[1900:2400]
postsDf["is_event"] = pd.NA
postsDf["processed_json"]=pd.NA

def check_string(input_string):
    if any(word in input_string for word in ['yes', 'Yes', 'True', 'true']):
        return True
    elif any(word in input_string for word in ['no', 'No', 'False', 'false']):
        return False
    else:
        return False  # This handles cases where none of the words are found


cnt=0
for index, row in postsDf.iterrows():
    currentjson = f"'account': '{row['account']}'; 'caption': '{row['caption']}'; 'photo_caption': '{row['accessibility_caption']}; 'date_posted_on_instagram: {row['date']}'"
    postsDf.at[index, "processed_json"] = currentjson
    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        messages=[{"role": "user", "content": f'Does the following instagram post contain a club event with a specified time. RETURN Yes or No only: {currentjson}'}],
        max_tokens=2
    )
    print(response.choices[0].message.content)
    is_event = check_string(response.choices[0].message.content)
    postsDf.at[index, "is_event"] = is_event
    time.sleep(0.6)

Yes
Yes
Yes
No
No
Yes
Yes
Yes
No
Yes
No
Yes
No
Yes
Yes
Yes
No
Yes
No
Yes
Yes
Yes
Yes
No
Yes
Yes
No
Yes
Yes
No
Yes
Yes
No
No
No
Yes
Yes
Yes
Yes
Yes
No
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
Yes
No
No
Yes
Yes
Yes
No
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
No
No
No
No
No
Yes
No
No
No
Yes
No
No
No
No
No
No
No
Yes
Yes
Yes
No
Yes
Yes
No
No
No
Yes
Yes
No
No
No
Yes
No
No
No
No
No
No
No
No
Yes
No
No
No
No
No
No
Yes
No
Yes
No
No
No
No
Yes
No
No
No
No
No
No
Yes
No
No
No
No
No
No
No
No
No
No
No
No
No
Yes
No
No
No
No
No
No
No
No
No
No
No
No
No
No
No
No
No
Yes
No
No
No
No
No
Yes
No
Yes
No
No
No
No
No
No
No
No
No
No
No
No
No
No
No
No
No
Yes
No
No
No
No
No
No
No
Yes
No
No
No
No
No
No
Yes
No
No
No
No
No
No
No
No
No
Yes
No
No
No
Yes
No
No
No
No
No
No
No
No
Yes
No
Yes
No
No
No
Yes
No
Yes
No
No
No
No
No
No
Yes
No
No
Yes
No
No
No
Yes
No
Yes
No
Yes
No
Yes
No
No
No
No
No
Yes
Yes
No
No
Yes
No
No
Yes
No
Yes
No
No
No
Yes
No
No
No
No
Yes
No
No
No
No
No
No
No
No
No
No
No
Yes
No
Yes
No
No
No
Yes
N

In [75]:
#Functions to Aid in LLM JSON data extraction


client = Together(api_key=togetherAPI)


def remove_emojis(text):
    # Unicode ranges for emojis
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

class Category(Enum):
    TECH = auto()
    DESIGN = auto()
    SOCIAL = auto()
    ENTERTAINMENT = auto()
    CULTURE = auto()
    SPORTS = auto()
    NETWORKING = auto()
    GAMING = auto()

class Event(BaseModel):
    is_event: bool = Field(description="Whether the post contains an event")
    event_name: str = Field(description="The Name of the Event")
    event_description: str = Field(description='Concise 20 word summary of the event without time or location')
    event_categories: list[str] = Field(description='Categorize the Event into at least one or more of the following: TECH, DESIGN, SOCIAL, MUSIC, CULTURE, SPORTS, NETWORK, GAMING')
    start_time: str = Field(description="The Start time of Event in the format: yyyy-mm-ddTHH:MM:SS+00:00")
    end_time: str = Field(description="The End time of Event in the format: yyyy-mm-ddTHH:MM:SS+00:00")
    location: str = Field(description= "The location of event")

def return_event_details(inputJson : str):
    chat_completion = client.chat.completions.create(
    model = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    response_format={"type": "json_object", "schema": Event.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": basePrompt,
        },
        {
            "role": "user",
            "content": "\n\n Input " + remove_emojis(inputJson) + "\n\n" + "Output",
        },
    ])
    
    created_event = json.loads(chat_completion.choices[0].message.content)
    return created_event

def extract_details_with_error_handling(inputJson, index):
        try: 
            created_event = return_event_details(inputJson)
            return created_event
        except Exception as err:
            print(str(err))
            return {'is_event': False, 'event_name': None, 'start_time': None, 'end_time': None, 'location': None}

postsDf["event_details"] = pd.NA


In [76]:
import requests
import os
import sys
from pathlib import Path

def download_image(url: str, save_path: str) -> bool:
    """
    Download an image from a URL and save it to the specified path.
    
    Args:
        url (str): The URL of the image to download
        save_path (str): The path where the image should be saved
        
    Returns:
        bool: True if successful, False otherwise
    """
    try:
        # Convert to Path object for better path handling
        save_path = Path(save_path)
        
        # Ensure the filename has a valid extension
        if not save_path.suffix:
            save_path = save_path.with_suffix('.jpg')
            
        # Create the directory if it doesn't exist
        save_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Check write permissions for the directory
        if not os.access(str(save_path.parent), os.W_OK):
            print(f"Error: No write permission for directory: {save_path.parent}")
            return False
            
        # Check if file already exists
        if save_path.exists():
            print(f"Warning: File already exists at {save_path}")
            # You might want to implement a strategy here (skip, overwrite, rename)
            return False
            
        # Download the image
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Raises an HTTPError for bad responses
        except requests.exceptions.RequestException as e:
            print(f"Error downloading image: {e}")
            return False
            
        # Verify the content type is an image
        content_type = response.headers.get('content-type', '')
        if not content_type.startswith('image/'):
            print(f"Error: URL does not point to an image (content-type: {content_type})")
            return False
            
        # Save the image
        try:
            save_path.write_bytes(response.content)
            print(f"Image successfully downloaded to {save_path}")
            return True
        except IOError as e:
            print(f"Error saving image: {e}")
            return False
            
    except Exception as e:
        print(f"Unexpected error: {e}")
        return False


In [77]:
for index, row in postsDf.iterrows():
    if postsDf.at[index, "is_event"]== True:
        event_details = return_event_details(str(postsDf.at[index, "processed_json"]))
        print(index, event_details)
        try:
            if event_details["event_name"] == None or event_details["start_time"] == None or event_details["end_time"] == None:
                postsDf.at[index, "is_event"] = False
                postsDf.at[index, "event_details"] = None
            else:
                postsDf.at[index, "event_details"] = event_details
                imageurl = row["display_photo"]
                postID = row["url"]
                filepath = image_filepath + postID + ".jpg"
                download_image(imageurl, filepath)
        except Exception as err:
            print(err)
    else: print(index, "no event detected")

1900 {'is_event': False, 'event_name': '', 'event_description': '', 'event_categories': [], 'start_time': '', 'end_time': '', 'location': ''}
Image successfully downloaded to ..\public\InstagramImages\DArKJi8ytPm.jpg
1901 {'is_event': False, 'event_name': '', 'event_description': '', 'event_categories': [], 'start_time': '', 'end_time': '', 'location': ''}
Image successfully downloaded to ..\public\InstagramImages\DAo_KOAgSJw.jpg
1902 {'is_event': True, 'event_name': 'F24 CSC Projects Program Kickoff', 'event_description': 'Bring your ideas to life and collaborate with fellow UWaterloo students on exciting projects', 'event_categories': ['TECH'], 'start_time': '2024-10-04T19:00:00+00:00', 'end_time': '2024-10-04T21:00:00+00:00', 'location': 'MC 1085'}
Image successfully downloaded to ..\public\InstagramImages\DAjn-woOCSv.jpg
1903 no event detected
1904 no event detected
1905 {'is_event': False, 'event_name': '', 'event_description': '', 'event_categories': [], 'start_time': '', 'end_ti

In [78]:
#RAG Processing
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

#time conversion and edit formats
from datetime import datetime
import time

postsDf.reset_index(drop=True)
postsDf = postsDf.fillna(value="None")
postsDf.drop(postsDf.columns[postsDf.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True)

for index, row in postsDf.iterrows():
    if row["is_event"] == True:
      row_dict = row["event_details"]
      try:
        conStart = datetime.fromisoformat(row_dict["start_time"])
        unixStart = time.mktime(conStart.timetuple()) 
        try:
          conEnd = datetime.fromisoformat(row_dict["end_time"])
          unixEnd = time.mktime(conEnd.timetuple())
        except:
          unixEnd = None
        row_dict["start_time"]=unixStart
        row_dict["end_time"]=unixEnd
        postsDf.at[index, "event_details"] = row_dict
        print(unixStart,unixEnd)
      except:
        postsDf.at[index, "is_event"] = False
        err = f'Start failed {row_dict["start_time"]}'
        
together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'WhereIsAI/UAE-Large-V1' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

for index, row in postsDf.iterrows():    
    if row['event_details'] != "None":
      try:
        row_dict = {}
        for column in postsDf.columns.tolist():
          if column in ['account','date','caption','accessibility_caption','url','likes','display_photo']:
            row_dict[column] = row[column]
        row_dict["embedded"] = generate_embedding(row["processed_json"], embedding_model_string)  
        row_dict["event_details"] = row["event_details"]
        result = collection.insert_one(row_dict)
        print(f"Inserted document ID: {result.inserted_id}")
        time.sleep(1)
      except Exception as err:
         print("failure:", "err", row["processed_json"])

#Updating Events.json

def download_future_events_to_json(output_file):
    # Get current Unix timestamp
    current_timestamp = int(time.time())

    # Connect to MongoDB
    load_dotenv()
    uri = os.getenv('DATABASE_URI')
    client = MongoClient(uri, server_api=ServerApi('1'))
    db = client['Instagram']
    collection = db["Events"]

    # Query for future events
    query = {}
    future_events = list(collection.find(query))

    # Convert ObjectId to string for JSON serialization
    for event in future_events:
        event['_id'] = str(event['_id'])
        event['embedded'] = ""
    # Write to JSON file
    with open(output_file, 'w') as f:
        json.dump(future_events, f, indent=2)

    # Close the MongoDB connection
    client.close()

    return future_events


future_events = download_future_events_to_json(output_file)


1728082800.0 1728090000.0
1726786800.0 1726794000.0
1726873200.0 1726880400.0
1725922800.0 None
1721338200.0 1721341800.0
1721343600.0 1721350800.0
1720908000.0 None
1719543600.0 None
1719442800.0 1719450000.0
1718924400.0 1718931600.0
1718233200.0 1718240400.0
1718233200.0 1718240400.0
1718319600.0 1718326800.0
1717628400.0 1722472200.0
1717110000.0 1717117200.0
1715641200.0 None
1713025800.0 1713036600.0
1712268000.0 1712275200.0
1712185200.0 1712192400.0
1711598400.0 1711684799.0
1711215000.0 1711229400.0
1710457200.0 None
1710543600.0 1710550800.0
1678744800.0 1678755600.0
1709854200.0 1709861400.0
1709766000.0 1709773200.0
1709082000.0 1709096340.0
1708990200.0 1708993800.0
1709337600.0 1709344800.0
1708045200.0 1708052400.0
1707955200.0 1707962400.0
1707868800.0 1707876000.0
1706986800.0 1706997600.0
1706227200.0 1706230800.0
1706140800.0 1706148000.0
1705014000.0 None
1701471600.0 1701482400.0
1700780400.0 1700787600.0
1700694000.0 1700701200.0
1700526600.0 1700535600.0
17002656